<a href="https://colab.research.google.com/github/eduusouza/PCD/blob/main/Entrega_2_Trabalho_PCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código usando CUDA

## Configurando CUDA

In [ ]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

In [ ]:
!nvidia-smi

Fri Jan 17 00:41:47 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Execução da aplicação deviceQuery

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git
%cd cuda-samples/Samples/1_Utilities/deviceQuery
!make

Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (4227/4227), done.
remote: Compressing objects: 100% (655/655), done.
remote: Total 19507 (delta 3932), reused 3572 (delta 3572), pack-reused 15280 (from 2)
Receiving objects: 100% (19507/19507), 133.82 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (17105/17105), done.
Updating files: 100% (4026/4026), done.
/content/cuda-samples/Samples/1_Utilities/deviceQuery/cuda-samples/Samples/1_Utilities/deviceQuery
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -

## Código 1 thread p/ bloco

In [ ]:
%%writefile code1.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

#define N 4000  // Tamanho da grade
#define T 500   // Número de iterações no tempo
#define D 0.1   // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

// Kernel CUDA para calcular a nova matriz de concentração
__global__ void diff_eq_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        C_new[i * n + j] = C[i * n + j] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] +
             C[i * n + (j + 1)] + C[i * n + (j - 1)] -
             4.0 * C[i * n + j]) / (DELTA_X * DELTA_X)
        );

        atomicAdd(difmedio, fabs(C_new[i * n + j] - C[i * n + j]));
    }
}

// Função principal
int main() {
    size_t size = N * N * sizeof(double);

    // Alocar memória no host (CPU)
    double *C = (double *)malloc(size);
    double *C_new = (double *)malloc(size);

    if (C == NULL || C_new == NULL) {
        fprintf(stderr, "Falha na alocação de memória no host\n");
        return 1;
    }

    // Inicializar as matrizes no host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocar memória no dispositivo (GPU)
    double *d_C, *d_C_new, *d_difmedio;
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiar dados do host para o dispositivo
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);






    dim3 threadsPerBlock(1, 1);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Loop de iterações no tempo
    double start_time = clock();
    for (int t = 0; t < T; t++) {
        double difmedio = 0.0;
        cudaMemcpy(d_difmedio, &difmedio, sizeof(double), cudaMemcpyHostToDevice);

        // Executar o kernel
        diff_eq_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);
        cudaDeviceSynchronize();

        // Trocar os ponteiros
        double *temp = d_C;
        d_C = d_C_new;
        d_C_new = temp;

        // Obter difmedio do dispositivo
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0) {
            printf("Interação %d - diferença = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        }
    }
    double end_time = clock();

    // Copiar os resultados finais para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Concentração final no centro: %f\n", C[N / 2 * N + N / 2]);
    printf("Tempo de execução: %f segundos\n", (end_time - start_time) / CLOCKS_PER_SEC);

    // Liberar memória no dispositivo e no host
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Writing code1.cu


In [ ]:
!nvcc -arch=sm_60 code1.cu -o code1


In [ ]:
!./code1

Interação 0 - diferença = 5.005e-10
Interação 100 - diferença = 3.07812e-10
Interação 200 - diferença = 1.95253e-10
Interação 300 - diferença = 1.27754e-10
Interação 400 - diferença = 1.05302e-10
Concentração final no centro: 0.216512
Tempo de execução: 16.946408 segundos


## Código 4 thread p/ bloco

In [ ]:
%%writefile code2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

#define N 4000  // Tamanho da grade
#define T 500   // Número de iterações no tempo
#define D 0.1   // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

// Kernel CUDA para calcular a nova matriz de concentração
__global__ void diff_eq_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        C_new[i * n + j] = C[i * n + j] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] +
             C[i * n + (j + 1)] + C[i * n + (j - 1)] -
             4.0 * C[i * n + j]) / (DELTA_X * DELTA_X)
        );

        atomicAdd(difmedio, fabs(C_new[i * n + j] - C[i * n + j]));
    }
}

// Função principal
int main() {
    size_t size = N * N * sizeof(double);

    // Alocar memória no host (CPU)
    double *C = (double *)malloc(size);
    double *C_new = (double *)malloc(size);

    if (C == NULL || C_new == NULL) {
        fprintf(stderr, "Falha na alocação de memória no host\n");
        return 1;
    }

    // Inicializar as matrizes no host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocar memória no dispositivo (GPU)
    double *d_C, *d_C_new, *d_difmedio;
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiar dados do host para o dispositivo
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);








    dim3 threadsPerBlock(2, 2);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Loop de iterações no tempo
    double start_time = clock();
    for (int t = 0; t < T; t++) {
        double difmedio = 0.0;
        cudaMemcpy(d_difmedio, &difmedio, sizeof(double), cudaMemcpyHostToDevice);

        // Executar o kernel
        diff_eq_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);
        cudaDeviceSynchronize();

        // Trocar os ponteiros
        double *temp = d_C;
        d_C = d_C_new;
        d_C_new = temp;

        // Obter difmedio do dispositivo
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0) {
            printf("Interação %d - diferença = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        }
    }
    double end_time = clock();

    // Copiar os resultados finais para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Concentração final no centro: %f\n", C[N / 2 * N + N / 2]);
    printf("Tempo de execução: %f segundos\n", (end_time - start_time) / CLOCKS_PER_SEC);

    // Liberar memória no dispositivo e no host
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Writing code2.cu


In [ ]:
!nvcc -arch=sm_60 code2.cu -o code2

In [ ]:
!./code2

Interação 0 - diferença = 5.005e-10
Interação 100 - diferença = 3.07812e-10
Interação 200 - diferença = 1.95253e-10
Interação 300 - diferença = 1.27754e-10
Interação 400 - diferença = 1.05302e-10
Concentração final no centro: 0.216512
Tempo de execução: 15.849324 segundos


## Código 16 thread p/ bloco

In [ ]:
%%writefile code3.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

#define N 4000  // Tamanho da grade
#define T 500   // Número de iterações no tempo
#define D 0.1   // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

// Kernel CUDA para calcular a nova matriz de concentração
__global__ void diff_eq_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        C_new[i * n + j] = C[i * n + j] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] +
             C[i * n + (j + 1)] + C[i * n + (j - 1)] -
             4.0 * C[i * n + j]) / (DELTA_X * DELTA_X)
        );

        atomicAdd(difmedio, fabs(C_new[i * n + j] - C[i * n + j]));
    }
}

// Função principal
int main() {
    size_t size = N * N * sizeof(double);

    // Alocar memória no host (CPU)
    double *C = (double *)malloc(size);
    double *C_new = (double *)malloc(size);

    if (C == NULL || C_new == NULL) {
        fprintf(stderr, "Falha na alocação de memória no host\n");
        return 1;
    }

    // Inicializar as matrizes no host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocar memória no dispositivo (GPU)
    double *d_C, *d_C_new, *d_difmedio;
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiar dados do host para o dispositivo
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);








    dim3 threadsPerBlock(4, 4);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Loop de iterações no tempo
    double start_time = clock();
    for (int t = 0; t < T; t++) {
        double difmedio = 0.0;
        cudaMemcpy(d_difmedio, &difmedio, sizeof(double), cudaMemcpyHostToDevice);

        // Executar o kernel
        diff_eq_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);
        cudaDeviceSynchronize();

        // Trocar os ponteiros
        double *temp = d_C;
        d_C = d_C_new;
        d_C_new = temp;

        // Obter difmedio do dispositivo
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0) {
            printf("Interação %d - diferença = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        }
    }
    double end_time = clock();

    // Copiar os resultados finais para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Concentração final no centro: %f\n", C[N / 2 * N + N / 2]);
    printf("Tempo de execução: %f segundos\n", (end_time - start_time) / CLOCKS_PER_SEC);

    // Liberar memória no dispositivo e no host
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Writing code3.cu


In [ ]:
!nvcc -arch=sm_60 code3.cu -o code3


In [ ]:
!./code3

Interação 0 - diferença = 5.005e-10
Interação 100 - diferença = 3.07812e-10
Interação 200 - diferença = 1.95253e-10
Interação 300 - diferença = 1.27754e-10
Interação 400 - diferença = 1.05302e-10
Concentração final no centro: 0.216512
Tempo de execução: 15.876949 segundos


## Código 64 thread p/ bloco

In [ ]:
%%writefile code4.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

#define N 4000  // Tamanho da grade
#define T 500   // Número de iterações no tempo
#define D 0.1   // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

// Kernel CUDA para calcular a nova matriz de concentração
__global__ void diff_eq_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        C_new[i * n + j] = C[i * n + j] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] +
             C[i * n + (j + 1)] + C[i * n + (j - 1)] -
             4.0 * C[i * n + j]) / (DELTA_X * DELTA_X)
        );

        atomicAdd(difmedio, fabs(C_new[i * n + j] - C[i * n + j]));
    }
}

// Função principal
int main() {
    size_t size = N * N * sizeof(double);

    // Alocar memória no host (CPU)
    double *C = (double *)malloc(size);
    double *C_new = (double *)malloc(size);

    if (C == NULL || C_new == NULL) {
        fprintf(stderr, "Falha na alocação de memória no host\n");
        return 1;
    }

    // Inicializar as matrizes no host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocar memória no dispositivo (GPU)
    double *d_C, *d_C_new, *d_difmedio;
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiar dados do host para o dispositivo
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);








    dim3 threadsPerBlock(8, 8);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Loop de iterações no tempo
    double start_time = clock();
    for (int t = 0; t < T; t++) {
        double difmedio = 0.0;
        cudaMemcpy(d_difmedio, &difmedio, sizeof(double), cudaMemcpyHostToDevice);

        // Executar o kernel
        diff_eq_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);
        cudaDeviceSynchronize();

        // Trocar os ponteiros
        double *temp = d_C;
        d_C = d_C_new;
        d_C_new = temp;

        // Obter difmedio do dispositivo
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0) {
            printf("Interação %d - diferença = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        }
    }
    double end_time = clock();

    // Copiar os resultados finais para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Concentração final no centro: %f\n", C[N / 2 * N + N / 2]);
    printf("Tempo de execução: %f segundos\n", (end_time - start_time) / CLOCKS_PER_SEC);

    // Liberar memória no dispositivo e no host
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Writing code4.cu


In [ ]:
!nvcc -arch=sm_60 code4.cu -o code4


In [ ]:
!./code4

Interação 0 - diferença = 5.005e-10
Interação 100 - diferença = 3.07812e-10
Interação 200 - diferença = 1.95253e-10
Interação 300 - diferença = 1.27754e-10
Interação 400 - diferença = 1.05302e-10
Concentração final no centro: 0.216512
Tempo de execução: 15.884956 segundos


## Código 256 thread p/ bloco

In [ ]:
%%writefile code5.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>

#define N 4000  // Tamanho da grade
#define T 500   // Número de iterações no tempo
#define D 0.1   // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

// Kernel CUDA para calcular a nova matriz de concentração
__global__ void diff_eq_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        C_new[i * n + j] = C[i * n + j] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] +
             C[i * n + (j + 1)] + C[i * n + (j - 1)] -
             4.0 * C[i * n + j]) / (DELTA_X * DELTA_X)
        );

        atomicAdd(difmedio, fabs(C_new[i * n + j] - C[i * n + j]));
    }
}

// Função principal
int main() {
    size_t size = N * N * sizeof(double);

    // Alocar memória no host (CPU)
    double *C = (double *)malloc(size);
    double *C_new = (double *)malloc(size);

    if (C == NULL || C_new == NULL) {
        fprintf(stderr, "Falha na alocação de memória no host\n");
        return 1;
    }

    // Inicializar as matrizes no host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocar memória no dispositivo (GPU)
    double *d_C, *d_C_new, *d_difmedio;
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiar dados do host para o dispositivo
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);








    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Loop de iterações no tempo
    double start_time = clock();
    for (int t = 0; t < T; t++) {
        double difmedio = 0.0;
        cudaMemcpy(d_difmedio, &difmedio, sizeof(double), cudaMemcpyHostToDevice);

        // Executar o kernel
        diff_eq_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);
        cudaDeviceSynchronize();

        // Trocar os ponteiros
        double *temp = d_C;
        d_C = d_C_new;
        d_C_new = temp;

        // Obter difmedio do dispositivo
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0) {
            printf("Interação %d - diferença = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        }
    }
    double end_time = clock();

    // Copiar os resultados finais para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Concentração final no centro: %f\n", C[N / 2 * N + N / 2]);
    printf("Tempo de execução: %f segundos\n", (end_time - start_time) / CLOCKS_PER_SEC);

    // Liberar memória no dispositivo e no host
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Writing code5.cu


In [ ]:
!nvcc -arch=sm_60 code5.cu -o code5


In [ ]:
!./code5

Interação 0 - diferença = 5.005e-10
Interação 100 - diferença = 3.07812e-10
Interação 200 - diferença = 1.95253e-10
Interação 300 - diferença = 1.27754e-10
Interação 400 - diferença = 1.05302e-10
Concentração final no centro: 0.216512
Tempo de execução: 15.873758 segundos
